In [123]:
import time
import requests
import pandas as pd
import datetime
import os
from barcelona_keys import key as weather_key
from functions_barcelona import scraping_weather, creating_datetime, creating_yearly_weather, mes_english_number
%load_ext autoreload
%reload_ext autoreload
%autoreload 2
print('DONE')

pathname='./data/weather/'

renaming_columns={'PeríodeTU':'period_UT',
                  'TM°C':'temp_avg',
                  'TX°C':'temp_max',
                  'TN°C':'temp_min',
                  'HRM%': 'relative_humidity',
                  'PPTmm':'precipitation',
                  'VVM (10 m)km/h': 'windspeed',
                   'DVM (10 m)graus': 'wind_direction',
                  'VVX (10 m)km/h' : 'max_windspeed',
                  'PMhPa': 'pressure',
                  'RSW/m2':'solar_radiation'}



# Bunch of years
first_year=2010
last_year=2023
for year in range(first_year,last_year+1):
    creating_yearly_weather(year,pathname)

##pending to convert to Europe/Brussels. Everything is in UT time up until I join it with accidents
##AT that point I will convert it doing:#df['datetime']=df['datetime'].dt.tz_localize('utc').dt.tz_convert('Europe/Brussels')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
DONE
Done with 2010
Done with 2011
Done with 2012
Done with 2013
Done with 2014
Done with 2015
Done with 2016
Done with 2017
Done with 2018
Done with 2019
Done with 2020
Done with 2021
Done with 2022
Done with 2023


In [124]:
#Puttin togetheraccidents and weather
total_weather=pd.read_csv('./data/weather/weatherfinal.csv',low_memory=False)
total_weather.drop('index',axis=1,inplace=True)

def fixing_datetime_format(row):
    '''Form some reason, datetime for some year is in utc format while the rest are not'''
    return row['datetime'][:19]


total_weather['datetime']=total_weather.apply(fixing_datetime_format,axis=1)
total_weather['datetime']=pd.to_datetime(total_weather.datetime).dt.tz_localize('utc').dt.tz_convert('Europe/Brussels')
#total_weather.interpolate(inplace=True)
total_weather.head()
print('NULLS: ',total_weather.isnull().sum().sum())
total_weather.replace({'(s/d)':None},inplace=True)
columns_w_nulls=list(total_weather.isnull().sum()[total_weather.isnull().sum()>0].index)
for col in columns_w_nulls:
    total_weather[col]=total_weather[col].astype(float).interpolate()
print('NULLS: ',total_weather.isnull().sum().sum())
acc=pd.read_csv('./data/accidents2023.csv')

def creating_datetime(row):
    return str(int(row['year']))+'-'+mes_english_number(row['month'])+'-'+str(int(row['day']))+' '+str(int(row['hour']))+':00:00'
acc['datetime']=acc.apply(creating_datetime,axis=1)
acc['datetime']=pd.to_datetime(acc.datetime).dt.tz_localize('utc').dt.tz_convert('Europe/Brussels')

pd.merge(acc,total_weather,how='left',on='datetime').to_csv('./data/accidents_weather.csv',index=False)



NULLS:  1008
NULLS:  0
